In [1]:
import pandas as pd 
#import geopandas as gp
import numpy as np
#import altair as alt 
#import gpdvega 
import json
import matplotlib.pyplot as plt
from ast import literal_eval
#alt.renderers.enable('notebook') # render for Jupyter Notebook

## Replace following variables with appropriate inputs

In [2]:
TURNSTILE_TO_OUTAGE_DATA = '/nas/dft/ire/susanw/out/turnstile_to_outage_elevations.csv'
TURNSTILE_RAW_DATA = '/nas/dft/ire/kmohan/hackday_2019_q4_mta_accessibility/data/turnstile_new.pkl.gz'
START_DATE = '20190101'
OUTPUT = '/nas/dft/ire/jujiang/DataClinic2019Q4HackDay/hourly_turnstile.csv.gz'
OLD_DATA= None # if exists

## Read the data for turnstiles with equipments

In [3]:
turnstile_to_outage = pd.read_csv(TURNSTILE_TO_OUTAGE_DATA)
stations_with_equipments = turnstile_to_outage.Turnstile_station.unique()

## Read all turnstile data and filter

In [4]:
turnstile = pd.read_pickle(TURNSTILE_RAW_DATA)
filtered_turnstile = turnstile.loc[(turnstile.entry_diff > 0)
                                    & (turnstile.exit_diff > 0)
                                    & (turnstile.STATION.isin(stations_with_equipments))]
filtered_turnstile.reset_index(inplace=True)
filtered_turnstile = filtered_turnstile.loc[filtered_turnstile['datetime'] > pd.to_datetime(START_DATE, format='%Y%m%d', errors='ignore')]

## Aggregate turnstile data by LINENAME, STATION and datetime

In [13]:
def interpolate(data):
    augmented = data.resample('1H').asfreq()
    augmented.entry_diff.interpolate(method='linear', inplace=True)
    augmented.exit_diff.interpolate(method='linear', inplace=True)
    augmented.fillna(method='ffill', inplace=True)
    return augmented

In [6]:
filtered_turnstile = filtered_turnstile.groupby(['LINENAME', 'STATION', 'UNIT', 'SCP', 'datetime']).sum().reset_index()

In [19]:
filtered_turnstile.set_index(pd.DatetimeIndex(filtered_turnstile.datetime), inplace=True)

In [27]:
dfs = []
filtered_turnstile.groupby(['LINENAME', 'STATION', 'UNIT', 'SCP']).apply(lambda g: dfs.append(interpolate(g)))
interpolated = pd.concat(dfs)

2020-01-06 02:31:36.756296
2020-01-06 02:32:26.255091


## Concatenate with existed data

In [ ]:
if OLD_DATA:
    interpolated = pd.concat(pd.read_pickle(OLD_DATA), interpolated)

## Save the data

In [ ]:
interpolated.to_csv(OUTPUT, compression='gzip')